Resources Used
- wget.download('https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py')
- Setup https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html

  Video https://www.youtube.com/watch?v=IOI0o3Cxv9Q

# 0. Setup Paths

In [5]:
import os
os.path

<module 'ntpath' from 'C:\\ProgramData\\Anaconda3\\envs\\Mask_Detection_Nic\\lib\\ntpath.py'>

In [6]:
import tensorflow as tf
tf.__version__

'2.5.0'

In [7]:
%matplolib inline

UsageError: Line magic function `%matplolib` not found.


In [8]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

# 1. Create Label Map.pbtxt


In [10]:
labels = [{'name':'Masked', 'id':1}, {'name':'No Mask', 'id':2}]

with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item{\n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 2. Create TF records from generate_tfrecord.py


In [9]:
SCRIPTS_PATH,  IMAGE_PATH,   ANNOTATION_PATH

('Tensorflow/scripts',
 'Tensorflow/workspace/images',
 'Tensorflow/workspace/annotations')

In [12]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x{IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


# 3. Download TF Models Pretrained Models from Tensorflow Model Zoo

!! Carefull !!   1.6 GB file will be clone from github. instead copy models folder from another env if u have n paste here.

In [1]:
YN = input("Carefull !! 1.6 GB file will be cloned from github... Enter 'y' to continue if you havent clonned before else 'n' to skip")
if YN=='y':
    print('clonning')
    !cd Tensorflow && git clone https://Github.com/tensorflow.org/models/
else:
    print('Skipped')

Carefull !! 1.6 GB file will be cloned from github... Enter 'y' to continue if you havent clonned before else 'n' to skipn
Skipped


In [61]:
#wget.download('http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz')
#!mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz {PRETRAINED_MODEL_PATH}
#!cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

'mv' is not recognized as an internal or external command,
operable program or batch file.


# 4. Copy Model Config to Training Folder

config.py file is file in which No_of_Epochs can be changed for training, steps are given, LR, optimizers can be changed.

In [9]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 

In [14]:
!mkdir {'Tensorflow\workspace\models\\'+CUSTOM_MODEL_NAME}
!cp {PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH+'/'+CUSTOM_MODEL_NAME}

A subdirectory or file Tensorflow\workspace\models\my_ssd_mobnet already exists.
'cp' is not recognized as an internal or external command,
operable program or batch file.


--Note-- cp command says copy pipeline.config from 'PRETRAINED_MODEL_PATH+'/ssd_mobile..' and paste in {MODEL_PATH+'/'+CUSTOM_MODEL_NAME}
try to run cp (!linux command) command in Git_Bash

# 5. Update Config For Transfer Learning

In [16]:
import os
os.getcwd()

'C:\\Users\\patil\\Downloads\\Deep_Learning\\Advance_DL\\Mask_Detection_Nic\\RealTimeObjectDetection'

os.chdir("C:\\Users\\patil\\Downloads\\Deep Learning\\Advance_DL\\Mask_Detection_N\\RealTimeObjectDetection\\Tensorflow\\models\\research")

!protoc object_detection/protos/*.proto --python_out=.

!copy(object_detection/packages/tf2/setup.py)

In [10]:
import tensorflow as tf
#from Tensorflow.models.research import object_detection
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

os.chdir("C:\\Users\\patil\\Downloads\\Deep Learning\\Advance_DL\\Mask_Detection_N\\RealTimeObjectDetection")

In [11]:
MODEL_PATH

'Tensorflow/workspace/models'

In [12]:
CUSTOM_MODEL_NAME

'my_ssd_mobnet'

In [13]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'
CONFIG_PATH

'Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config'

In [14]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [15]:
config

{'model': ssd {
   num_classes: 2
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   match

It says this'config' file built for 90 classes, Classification type problems(line 171), epichs n all..
#### but we want to use this for detection not for classificattion. 
####  So instead of doing this mannually in config file we are gonna do this with python script called  'pipeline_pb2' heance we have imported it.

In [16]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [17]:
pipeline_config.model.ssd.num_classes = 2
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [18]:
pipeline_config

model {
  ssd {
    num_classes: 2
    image_resizer {
      fixed_shape_resizer {
        height: 320
        width: 320
      }
    }
    feature_extractor {
      type: "ssd_mobilenet_v2_fpn_keras"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 3.9999998989515007e-05
          }
        }
        initializer {
          random_normal_initializer {
            mean: 0.0
            stddev: 0.009999999776482582
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.996999979019165
          scale: true
          epsilon: 0.0010000000474974513
        }
      }
      use_depthwise: true
      override_base_feature_extractor_hyperparams: true
      fpn {
        min_level: 3
        max_level: 7
        additional_layer_depth: 128
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
 

    As u can see that num_classes is now just 2 (not 90) as we have modified this file.
    slly we have changed Batch_soze = 4, 
    slly we have done fine_tune_checkpoint_type= "detection" instead of default "classification" and so on...



In [19]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 6. Train the model

!pip install tensorflow.compat.v2


import tensorflow.compat.v2 as tf

In [27]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=5000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=5000


before hitting this comand make sure u r in ReaTimeObjectDetection folder dir

copy the above command ( for training on 5000 steps) and paste in AnacondaPromt in ReaTimeObjectDetection folder directory.
when u hit enter Training will start at backend n u can still use ur jupyter notebook side-by-side


# 7. Load Train Model From Checkpoint

!pip install tf-models-official

In [20]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [21]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-3')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 8. Detect in Real-Time

In [22]:
import cv2 
import numpy as np

In [23]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')
category_index

{1: {'id': 1, 'name': 'Masked'}, 2: {'id': 2, 'name': 'No Mask'}}

In [24]:
# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [25]:
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=1,
                min_score_thresh=.3,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break

In [39]:
cap.release()

In [42]:
detections = detect_fn(input_tensor)

In [67]:
from matplotlib import pyplot as plt